In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import random
import cv2
import tensorflow as tf
import pydicom
from math import ceil, floor
from copy import deepcopy
from tqdm.notebook import tqdm
from imgaug import augmenters as iaa
from sklearn.utils import shuffle
import tensorflow.keras as keras
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import Callback, ModelCheckpoint
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import Sequence
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC, Recall, Precision, BinaryCrossentropy
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.layers import *
from sklearn.utils.class_weight import compute_class_weight

from prettytable import PrettyTable
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns



def correct_dcm(dcm):
    x = dcm.pixel_array + 1000
    px_mode = 4096
    x[x>=px_mode] = x[x>=px_mode] - px_mode
    dcm.PixelData = x.tobytes()
    dcm.RescaleIntercept = -1000

def window_image(dcm, window_center, window_width):    
    if (dcm.BitsStored == 12) and (dcm.PixelRepresentation == 0) and (int(dcm.RescaleIntercept) > -100):
        correct_dcm(dcm)
    img = dcm.pixel_array * dcm.RescaleSlope + dcm.RescaleIntercept
    img = cv2.resize(img, SHAPE[:2], interpolation = cv2.INTER_LINEAR)
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    img = np.clip(img, img_min, img_max)
    return img

def bsb_window(dcm):
    brain_img = window_image(dcm, 40, 80)
    subdural_img = window_image(dcm, 80, 200)
    brain_img = (brain_img - 0) / 80
    subdural_img = (subdural_img - (-20)) / 200
    soft_img = window_image(dcm, 40, 380)
    soft_img = (soft_img - (-150)) / 380
    bsb_img = np.array([brain_img, subdural_img, soft_img]).transpose(1,2,0)
    return bsb_img





def calculating_class_weights(y_true):    #  input : true labels  
                                        # output : weights of each class
    number_dim = np.shape(y_true)[1]
    weights = np.empty([number_dim, 2])
    for i in range(number_dim):
        weights[i] = compute_class_weight('balanced', classes=np.unique(y_true[:, i]), y=y_true[:, i])
    return weights


def _read(path, SHAPE):        # input : path of specific image and the shape that we want convert image to.
                              # output : readed image/255
    img = cv2.imread(path)
    img = cv2.resize(img, dsize=(256, 256))
    return img/255.0

def _read_dicom(path, SHAPE):        # input : the path of dicom image and its shape.
                                  # output : image in numpy format.
    dcm = pydicom.dcmread(path)
    try:
        img = bsb_window(dcm)
    except:
        img = np.zeros(SHAPE)
    return img

# Image Augmentation
sometimes = lambda aug: iaa.Sometimes(0.25, aug)

augmentation = iaa.Sequential([ iaa.Fliplr(0.25),
                                iaa.Flipud(0.10),
                                sometimes(iaa.Crop(px=(0, 25), keep_size = True, sample_independently = False))   
                            ], random_order = True)       
        
# Generators
class TrainDataGenerator(keras.utils.Sequence):
    def __init__(self, dataset, class_names, batch_size = 16, img_size = (256, 256, 3), 
                 augment = False, shuffle = True, *args, **kwargs):            # initialize datagenerator
        self.dataset = dataset
        self.ids = self.dataset['imgfile'].values
        self.labels = self.dataset[class_names].values
        self.batch_size = batch_size
        self.img_size = img_size
        self.augment = augment
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):        # size of datagenerator (number of batchs)
        return int(ceil(len(self.ids) / self.batch_size))

    def __getitem__(self, index):          # input : index of a batch
                                    # output : specific batch with input index
        indices = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        X, Y = self.__data_generation(indices)
        return X, Y

    def augmentor(self, image):        # input : image
                                 # output : augmented image
        augment_img = augmentation        
        image_aug = augment_img.augment_image(image)
        return image_aug

    def on_epoch_end(self):
        self.indices = np.arange(len(self.ids))
        if self.shuffle:
            np.random.shuffle(self.indices)

    def __data_generation(self, indices):        # creating augmented images and their labels.
                                          # input : /////////////////////////////////////
                                          # output : augmented images and their labels.
        X = np.empty((self.batch_size, *self.img_size))
        Y = np.empty((self.batch_size, len(class_names)), dtype=np.float32)
        
        for i, index in enumerate(indices):
            ID = self.ids[index]
#             image = _read(ID, self.img_size)
            if '.png' not in ID:
                image = _read_dicom('../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/stage_2_train/'+ID+'.dcm', self.img_size)
            else:
## just CQ500

                if 'NonHemo' in ID:
                        image = _read('../input/cq500-normal-images-and-labels/'+ID, self.img_size)
                else:
                        if 'content' in ID:
                            ID = ID[9:]
                        
                        image = _read('../input/rsna-cq500-abnormal-data/'+ID, self.img_size)
                        
            if self.augment:
                X[i,] = self.augmentor(image)
            else:
                X[i,] = image
            Y[i,] = self.labels[index]        
        return X, Y

def ModelCheckpointFull(model_name):        # save weights of the model that has the best result.
    return ModelCheckpoint(model_name, 
                            monitor = 'val_loss', 
                            verbose = 1, 
                            save_best_only = True, 
                            save_weights_only = True, 
                            mode = 'min', 
                            period = 1)

# Create Model
def create_model(num_classes):        # input : num of classes
                               # output : pretrained densenet121 to classify num_classes classes.
    K.clear_session()
    
    input_shape = (256, 256, 3)
    img_input = Input(shape=input_shape)
    base_model = DenseNet121(
        include_top=False,
        input_tensor=img_input,
        input_shape=input_shape,
        weights='imagenet',
        pooling="avg"
    )
    x = base_model.output
    x = Dropout(0.15)(x)
    predictions = Dense(num_classes, activation='sigmoid', name="new_predictions")(x)
    model = Model(inputs=img_input, outputs=predictions)

    return model

def metrics_define(num_classes):
    metrics_all = ['accuracy',
    AUC(curve='PR',multi_label=True,name='auc_pr'),
    AUC(multi_label=True, name='auc_roc'),
    Recall(),
    Precision(),
    BinaryCrossentropy(name='bi_crent')
    ]

    return metrics_all

def get_weighted_loss(weights):        # input : weights of classes
                                   # output : loss (weighted loss)
    def weighted_loss(y_true, y_pred):
        return K.mean((weights[:,0]**(1-y_true))*(weights[:,1]**(y_true))*K.binary_crossentropy(y_true, y_pred), axis=-1)
    return weighted_loss


def print_metrics_table(y_true, y_hat, y_pred, class_names):         # input : true labels, predicted labels, array of class names, y hat 
                                                             # output : a table of results (roc_auc_score, precision_score, f1_score, recall_xcore, accuracy_score)
    myTable = PrettyTable(["Class Name", "ROC_AUC", "Precsion", "Recall", "F1_Score", "Accuracy"])

    for i in range(len(class_names)) :
        
        myTable.add_row([class_names[i], "%.4f" % roc_auc_score(y_true[:, i], y_hat[:, i]),
                        "%.4f" % precision_score(y_true[:, i], y_pred[:, i]), "%.4f" % recall_score(y_true[:, i], y_pred[:, i]),
                        "%.4f" % f1_score(y_true[:, i], y_pred[:, i]), "%.4f" % accuracy_score(y_true[:, i], y_pred[:, i])
                        ])

    myTable.add_row(['Average', "%.4f" % roc_auc_score(y_true, y_hat),
                    "%.4f" % precision_score(y_true, y_pred, average='macro'), "%.4f" % recall_score(y_true, y_pred, average='macro'),
                    "%.4f" % f1_score(y_true, y_pred, average='macro'), "%.4f" % accuracy_score(y_true, y_pred)
                    ])
    print(myTable)

def print_precision_recall_curves(y_true, y_hat, y_pred, class_names):   # input : true labels, predicted labels, array of class names, y hat 
                                                                        # print precision and recall curves
        # For each class
    precision = dict()
    recall = dict()
    average_precision = dict()
    for i in range(len(class_names)):
        precision[i], recall[i], _ = precision_recall_curve(y_true[:, i],
                                                            y_hat[:, i])
        average_precision[i] = average_precision_score(y_true[:, i], y_hat[:, i])

    # A "micro-average": quantifying score on all classes jointly
    precision["micro"], recall["micro"], _ = precision_recall_curve(y_true.ravel(),
        y_hat.ravel())
    average_precision["micro"] = average_precision_score(y_true, y_hat,
                                                        average="micro")
    print('Average precision score, micro-averaged over all classes: {0:0.2f}'
        .format(average_precision["micro"]))
    plt.figure()
    plt.step(recall['micro'], precision['micro'], where='post')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    plt.title(
        'Average precision score, micro-averaged over all classes: AP={0:0.2f}'
        .format(average_precision["micro"]))
    plt.show()
    for i in range(len(class_names)):
        plt.figure()
        plt.plot(recall[i], precision[i], label='Precision-recall for class {0} (area = {1:0.2f})'.format(i, average_precision[i]))
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.title('Precision-Recall Curve for Class {}'.format(class_names[i]))
        plt.legend(loc="lower right")
        plt.show()

def print_auc_curves(y_true, y_hat, y_pred, class_names):    # input : true labels, predicted labels, array of class names, y hat 
                                                           # print auc curves
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    roc_auc_sc = dict()
    for i in range(len(class_names)):
        fpr[i], tpr[i], _ = roc_curve(y_true[:, i], y_hat[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
        roc_auc_sc[i] = roc_auc_score(y_true[:, i], y_hat[:, i])

    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_true.ravel(), y_hat.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
    for i in range(len(class_names)):
        plt.figure()
        plt.plot(fpr[i], tpr[i], label='ROC curve (area = %0.2f)' % roc_auc[i])
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver operating characteristic for class {}'.format(class_names[i]))
        plt.legend(loc="lower right")
        plt.show()

def print_confusion_matrix(y_true, y_hat, y_pred, class_names):        # input : true labels, predicted labels, array of class names, y hat 
                                                                # print confusion matrixes.
    print(multilabel_confusion_matrix(y_true, y_pred))

def print_on_vs_all_cmatrix(y_true, y_hat, y_pred, class_names):    # input : true labels, predicted labels, array of class names, y hat
                                                                  # print confusion matrix plots.
    confusion = multilabel_confusion_matrix(y_true, y_pred)

    # Plot confusion matrix 
    fig = plt.figure(figsize = (14, 8))
    for i, (label, matrix) in enumerate(zip(class_names[0:6], confusion[0:6])):
        plt.subplot(f'23{i+1}')
        labels = [f'not_{label}', label]
        cm = matrix.astype('float') / matrix.sum(axis=1)[:, np.newaxis]
        sns.heatmap(cm, annot = True, square = True, cbar = False, cmap = 'Blues', 
                    xticklabels = labels, yticklabels = labels, linecolor = 'black', linewidth = 1)
        plt.title(labels[0])

    plt.tight_layout()
    plt.show()

creating train and validation dataframes on fold 4

In [ ]:
train_df = pd.read_csv('../input/kfold-splits-rsna-cq500/Train_f4.csv')
val_df = pd.read_csv('../input/kfold-splits-rsna-cq500/Validation_f4.csv')
train_df = pd.DataFrame( train_df.loc[i][2:] for i in range(0, len(train_df)) if 'CQ500' in train_df.loc[i]['imgfile']  )    # just for CQ500 images.
val_df = pd.DataFrame( val_df.loc[i][2:] for i in range(0, len(val_df)) if 'CQ500' in val_df.loc[i]['imgfile']  ) 


In [ ]:
train_df

In [ ]:
val_df

In [ ]:
train_df['normal']=0   # add normal label to dataframes
val_df['normal']=0

In [ ]:
train_df

make dataframes for normal images.

In [ ]:

train_df1 = pd.read_csv('../input/cq500-normal-images-and-labels/NormalAbnormal/NormalAbnormal/CQ500/Train_f1.csv')
train_df1 = pd.DataFrame( train_df1.loc[i][1:] for i in range(0, len(train_df1))   ) 
abnormal1 = train_df1.loc[train_df1['Abnormal']==1]

train_df1 = shuffle(train_df1)
val_df1 = pd.read_csv('../input/cq500-normal-images-and-labels/NormalAbnormal/NormalAbnormal/CQ500/Validation_f1.csv')
val_df1 = pd.DataFrame( val_df1.loc[i][1:] for i in range(0, len(val_df1))   ) 

In [ ]:
train_df1

In [ ]:
val_df1

In [ ]:
train_df2 = pd.DataFrame( train_df1.loc[i][:1] for i in range(0, 8000) if train_df1.loc[i]['Normal'].all()==True  ) 
val_df2= pd.DataFrame( val_df1.loc[i][:1] for i in range(0, 4000) if val_df1.loc[i]['Normal'].all()==True )  

In [ ]:
train_df2['epidural']=0      # assign 0 to other classes.
train_df2['intraparenchymal']=0
train_df2['intraventricular']=0	
train_df2['subarachnoid']=0	
train_df2['subdural']=0
train_df2['normal']=1
train_df2.rename(columns={'Normal': 'normal'}, inplace=True)


val_df2['epidural']=0
val_df2['intraparenchymal']=0
val_df2['intraventricular']=0	
val_df2['subarachnoid']=0	
val_df2['subdural']=0
val_df2['normal']=1
val_df2.rename(columns={'Normal': 'normal'}, inplace=True)

In [ ]:
train_df2

concate 2 dataframes and creating a dataframe for all classes.

In [ ]:
frames = [train_df, train_df2]
train_df = pd.concat(frames, ignore_index=True)
train_df=shuffle(train_df)

In [ ]:
frames = [val_df, val_df2]
val_df = pd.concat(frames, ignore_index=True)
val_df=shuffle(val_df)

In [ ]:
val_df

In [ ]:
train_df

creating data generator for training the model.

In [ ]:


HEIGHT = 256
WIDTH = 256
CHANNELS = 3
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 64
SHAPE = (HEIGHT, WIDTH, CHANNELS)

class_names = ['epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural','normal']

weights = calculating_class_weights((train_df[class_names].values).astype(np.float32))
print(weights)

data_generator_train = TrainDataGenerator(train_df,
                                          class_names,
                                          TRAIN_BATCH_SIZE,
                                          SHAPE,
                                          augment = True,
                                          shuffle = True)
data_generator_val = TrainDataGenerator(val_df,
                                        class_names, 
                                        VALID_BATCH_SIZE, 
                                        SHAPE,
                                        augment = False,
                                        shuffle = True
                                        )

TRAIN_STEPS = int(len(data_generator_train)/10)
print(TRAIN_STEPS)
Val_STEPS = int(len(data_generator_val)/10)
print(Val_STEPS)
LR = 5e-6



Model

In [ ]:
plt.imshow(data_generator_val[1][0][0])

In [ ]:
Metrics = metrics_define(len(class_names))

model = create_model(len(class_names))   


from keras.models import Model

model.summary()


In [ ]:

model.compile(optimizer = Adam(learning_rate = LR),
              loss = get_weighted_loss(weights),
              metrics = Metrics)

fit model

In [ ]:
history = model.fit(data_generator_train,
                    validation_data = data_generator_val,
                    validation_steps = Val_STEPS,
                    steps_per_epoch = TRAIN_STEPS,
                    epochs = 100,
                    callbacks = [ModelCheckpointFull('model_all_densenet_fold4_andNormal.h5')],
                    verbose = 1, workers=4
                    )

results

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['auc_pr'])
plt.plot(history.history['val_auc_pr'])
plt.title('model auc_precision')
plt.ylabel('auc_pr')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

validate the model

In [ ]:
model.load_weights('model_all_densenet_fold4_andNormal.h5')

In [ ]:


print(len(val_df))

HEIGHT = 256
WIDTH = 256
CHANNELS = 3
VALID_BATCH_SIZE = 64
SHAPE = (HEIGHT, WIDTH, CHANNELS)

class_names = ['epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural', 'normal']
data_generator_test = TrainDataGenerator(val_df,
                                        class_names, 
                                        VALID_BATCH_SIZE, 
                                        SHAPE,
                                        augment = False,
                                        shuffle = False
                                        )

y_true = val_df[class_names].values
y_hat = model.predict(data_generator_test, verbose=1)

y_hat = y_hat[0:len(y_true)]
y_pred = np.round(y_hat)

In [ ]:
val_df

In [ ]:
print_metrics_table(y_true, y_hat, y_pred, class_names)

In [ ]:
print_precision_recall_curves(y_true, y_hat, y_pred, class_names)

In [ ]:
print_auc_curves(y_true, y_hat, y_pred, class_names)

In [ ]:
print_confusion_matrix(y_true, y_hat, y_pred, class_names)

In [ ]:
print_on_vs_all_cmatrix(y_true, y_hat, y_pred, class_names)

test another dataset

In [ ]:
val_df2= pd.DataFrame( val_df1.loc[i][:1] for i in range(4000, 8000) if val_df1.loc[i]['Normal'].all()==True )  

In [ ]:

val_df2['epidural']=0
val_df2['intraparenchymal']=0
val_df2['intraventricular']=0	
val_df2['subarachnoid']=0	
val_df2['subdural']=0
val_df2['normal']=1
val_df2.rename(columns={'Normal': 'normal'}, inplace=True)

In [ ]:
val_df = pd.read_csv('../input/kfold-splits-rsna-cq500/CQ500_Validation_f0.csv')
val_df = pd.DataFrame( val_df.loc[i][1:] for i in range(0, len(val_df)) if 'CQ500' in val_df.loc[i]['imgfile']  ) 

frames = [val_df, val_df2]
val_df = pd.concat(frames, ignore_index=True)
val_df=shuffle(val_df)

val_df['normal']=0
print(len(val_df))

HEIGHT = 256
WIDTH = 256
CHANNELS = 3
VALID_BATCH_SIZE = 64
SHAPE = (HEIGHT, WIDTH, CHANNELS)

class_names = ['epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural', 'normal']
data_generator_test = TrainDataGenerator(val_df,
                                        class_names, 
                                        VALID_BATCH_SIZE, 
                                        SHAPE,
                                        augment = False,
                                        shuffle = False
                                        )


y_true = val_df[class_names].values
y_hat = model.predict(data_generator_test, verbose=1)

y_hat = y_hat[0:len(y_true)]
y_pred = np.round(y_hat)

In [ ]:
val_df

In [ ]:
# print_metrics_table(y_true, y_hat, y_pred, class_names)

In [ ]:
print_precision_recall_curves(y_true, y_hat, y_pred, class_names)

In [ ]:
# print_auc_curves(y_true, y_hat, y_pred, class_names)

In [ ]:
print_confusion_matrix(y_true, y_hat, y_pred, class_names)

In [ ]:
print_on_vs_all_cmatrix(y_true, y_hat, y_pred, class_names)

test model on Normal dataset


In [ ]:
val_df = pd.read_csv('../input/cq500-normal-images-and-labels/NormalAbnormal/NormalAbnormal/CQ500/Validation_f3.csv')

val_df = pd.DataFrame( val_df.loc[i][1:2] for i in range(0, len(val_df)) if val_df.loc[i]['Normal'].all()==True)
                         
val_df['epidural']=0
val_df['intraparenchymal']=0
val_df['intraventricular']=0	
val_df['subarachnoid']=0	
val_df['subdural']=0
val_df['normal']=1


In [ ]:


print(len(val_df))

HEIGHT = 256
WIDTH = 256
CHANNELS = 3
VALID_BATCH_SIZE = 64
SHAPE = (HEIGHT, WIDTH, CHANNELS)

class_names = ['epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural', 'normal']
data_generator_test = TrainDataGenerator(val_df,
                                        class_names, 
                                        VALID_BATCH_SIZE, 
                                        SHAPE,
                                        augment = False,
                                        shuffle = False
                                        )


y_true = val_df[class_names].values
y_hat = model.predict(data_generator_test, verbose=1)

y_hat = y_hat[0:len(y_true)]
y_pred = np.round(y_hat)

In [ ]:
val_df

In [ ]:
# print_metrics_table(y_true, y_hat, y_pred, class_names)

In [ ]:
print_precision_recall_curves(y_true, y_hat, y_pred, class_names)

In [ ]:
# print_auc_curves(y_true, y_hat, y_pred, class_names)

In [ ]:
print_confusion_matrix(y_true, y_hat, y_pred, class_names)

In [ ]:
print_on_vs_all_cmatrix(y_true, y_hat, y_pred, class_names)

Grad Cam

In [ ]:
model = create_model(6)
model.load_weights('./model_all_densenet_fold4_andNormal.h5')

In [ ]:
model.summary()

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [ ]:
model_builder = model
img_size = (256, 256)

decode_predictions = keras.applications.mobilenet.decode_predictions

last_conv_layer_name = "conv5_block16_concat"

In [ ]:
def get_img_array(img_path, size):
    
    img = keras.preprocessing.image.load_img(img_path, target_size=size)
    
    array = keras.preprocessing.image.img_to_array(img)
    
    array = np.expand_dims(array, axis=0)
    return array


def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    
    grads = tape.gradient(class_channel, last_conv_layer_output)

    
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

   
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [ ]:
val_df = pd.read_csv('../input/kfold-splits-rsna-cq500/Validation_f4.csv')
val_df = pd.DataFrame( val_df.loc[i][2:] for i in range(0, len(val_df)) if 'CQ500' in val_df.loc[i]['imgfile']  ) 
val_df['normal']=0

val_df2= pd.DataFrame( val_df1.loc[i][:1] for i in range(0, 4000) if val_df1.loc[i]['Normal'].all()==True )
val_df2['epidural']=0
val_df2['intraparenchymal']=0
val_df2['intraventricular']=0	
val_df2['subarachnoid']=0	
val_df2['subdural']=0
val_df2['normal']=1
val_df2.rename(columns={'Normal': 'normal'}, inplace=True)

frames = [val_df, val_df2]
val_df = pd.concat(frames, ignore_index=True)
val_df=shuffle(val_df)

val_df






In [ ]:
data_generator_test = TrainDataGenerator(val_df,
                                        class_names, 
                                        VALID_BATCH_SIZE, 
                                        SHAPE,
                                        augment = False,
                                        shuffle = False
                                        )

In [ ]:
plt.imshow(data_generator_test[1][0][0])
plt.savefig( "ok.jpg")

In [ ]:
img_array= data_generator_test[1][0][0]
img_array=img_array.reshape(1,256,256,3)
np.shape(img_array)

In [ ]:
preds = model.predict(img_array)

heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)

# Display heatmap
plt.matshow(heatmap)
plt.show()

In [ ]:
def save_and_display_gradcam(img, heatmap, cam_path="cam.jpg", alpha=0.4):
 
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

    # Save the superimposed image
    superimposed_img.save(cam_path)

    # Display Grad CAM
    display(Image(cam_path))


save_and_display_gradcam(img_array[0]*255, heatmap)

In [ ]:
plt.imshow(data_generator_test[1][0][1])

In [ ]:
!mkdir 0
!mkdir 1
!mkdir 2
!mkdir 3
!mkdir 4
!mkdir 5

In [ ]:
!rm -r 0
!rm -r 1
!rm -r 2
!rm -r 3
!rm -r 4
!rm -r 5

In [ ]:
import os
h=0
for i in range(0,10):
    for j in range (0,64):
        x = data_generator_test[i][0][j]
        x = x.reshape(1,256,256,3)
        np.shape(x)
        
        h+=1
        preds = model.predict(x)
        heatmap = make_gradcam_heatmap(x, model, last_conv_layer_name)
        p=os.path.join("./", str(np.argmax(preds)))
        if len(os.listdir(p)) < 21: 
            save_and_display_gradcam(x[0]*255, heatmap, str(np.argmax(preds))+"/"+"cam_"+str(len(os.listdir(p)))+".jpg")
            print("--- "+str(np.argmax(preds))+" :  "+ str(len(os.listdir(p))))
            

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
w = 256
h = 256
fig = plt.figure(figsize=(9, 13))
columns = 3
rows = 5

# prep (x,y) for extra plotting
xs = np.linspace(0, 2*np.pi, 60)  # from 0 to 2pi
ys = np.abs(np.sin(xs))           # absolute of sine

# ax enables access to manipulate each of subplots
ax = []
p=os.path.join("./","0",)
for i in range(columns*rows):

#     img = np.random.randint(10, size=(h,w))
    img = cv2.imread(p+'/'+'cam_'+str(i)+'.jpg')

    
    # create subplot and append to ax
    ax.append( fig.add_subplot(rows, columns, i+1) )
    ax[-1].set_title("prediction : 0")  # set title
    plt.imshow(img, )

# do extra plots on selected axes/subplots
# note: index starts with 0
ax[2].plot(xs, 3*ys)
ax[19].plot(ys**2, xs)

plt.show()  # finally, render the plot

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
w = 256
h = 256
fig = plt.figure(figsize=(9, 13))
columns = 4
rows = 5

# prep (x,y) for extra plotting
xs = np.linspace(0, 2*np.pi, 60)  # from 0 to 2pi
ys = np.abs(np.sin(xs))           # absolute of sine

# ax enables access to manipulate each of subplots
ax = []
p=os.path.join("./","1",)
for i in range(columns*rows):

#     img = np.random.randint(10, size=(h,w))
    img = cv2.imread(p+'/'+'cam_'+str(i)+'.jpg')

    
    # create subplot and append to ax
    ax.append( fig.add_subplot(rows, columns, i+1) )
    ax[-1].set_title("prediction : 1")  # set title
    plt.imshow(img, )

# do extra plots on selected axes/subplots
# note: index starts with 0
ax[2].plot(xs, 3*ys)
ax[19].plot(ys**2, xs)

plt.show()  # finally, render the plot

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
w = 256
h = 256
fig = plt.figure(figsize=(9, 13))
columns = 4
rows = 5

# prep (x,y) for extra plotting
xs = np.linspace(0, 2*np.pi, 60)  # from 0 to 2pi
ys = np.abs(np.sin(xs))           # absolute of sine

# ax enables access to manipulate each of subplots
ax = []
p=os.path.join("./","2",)
for i in range(columns*rows):

#     img = np.random.randint(10, size=(h,w))
    img = cv2.imread(p+'/'+'cam_'+str(i)+'.jpg')

    
    # create subplot and append to ax
    ax.append( fig.add_subplot(rows, columns, i+1) )
    ax[-1].set_title("prediction : 2")  # set title
    plt.imshow(img, )

# do extra plots on selected axes/subplots
# note: index starts with 0
ax[2].plot(xs, 3*ys)
ax[19].plot(ys**2, xs)

plt.show()  # finally, render the plot

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
w = 256
h = 256
fig = plt.figure(figsize=(9, 13))
columns = 4
rows = 5

# prep (x,y) for extra plotting
xs = np.linspace(0, 2*np.pi, 60)  # from 0 to 2pi
ys = np.abs(np.sin(xs))           # absolute of sine

# ax enables access to manipulate each of subplots
ax = []
p=os.path.join("./","3",)
for i in range(columns*rows):

#     img = np.random.randint(10, size=(h,w))
    img = cv2.imread(p+'/'+'cam_'+str(i)+'.jpg')

    
    # create subplot and append to ax
    ax.append( fig.add_subplot(rows, columns, i+1) )
    ax[-1].set_title("prediction : 3")  # set title
    plt.imshow(img, )

# do extra plots on selected axes/subplots
# note: index starts with 0
ax[2].plot(xs, 3*ys)
ax[19].plot(ys**2, xs)

plt.show()  # finally, render the plot

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
w = 256
h = 256
fig = plt.figure(figsize=(9, 13))
columns = 4
rows = 5

# prep (x,y) for extra plotting
xs = np.linspace(0, 2*np.pi, 60)  # from 0 to 2pi
ys = np.abs(np.sin(xs))           # absolute of sine

# ax enables access to manipulate each of subplots
ax = []
p=os.path.join("./","4",)
for i in range(columns*rows):

#     img = np.random.randint(10, size=(h,w))
    img = cv2.imread(p+'/'+'cam_'+str(i)+'.jpg')

    
    # create subplot and append to ax
    ax.append( fig.add_subplot(rows, columns, i+1) )
    ax[-1].set_title("prediction : 4")  # set title
    plt.imshow(img, )

# do extra plots on selected axes/subplots
# note: index starts with 0
ax[2].plot(xs, 3*ys)
ax[19].plot(ys**2, xs)

plt.show()  # finally, render the plot

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
w = 256
h = 256
fig = plt.figure(figsize=(9, 13))
columns = 4
rows = 5

# prep (x,y) for extra plotting
xs = np.linspace(0, 2*np.pi, 60)  # from 0 to 2pi
ys = np.abs(np.sin(xs))           # absolute of sine

# ax enables access to manipulate each of subplots
ax = []
p=os.path.join("./","5",)
for i in range(columns*rows):

#     img = np.random.randint(10, size=(h,w))
    img = cv2.imread(p+'/'+'cam_'+str(i)+'.jpg')

    
    # create subplot and append to ax
    ax.append( fig.add_subplot(rows, columns, i+1) )
    ax[-1].set_title("prediction : 5")  # set title
    plt.imshow(img, )

# do extra plots on selected axes/subplots
# note: index starts with 0
ax[2].plot(xs, 3*ys)
ax[19].plot(ys**2, xs)

plt.show()  # finally, render the plot

test on RSNA dataset

In [ ]:
rsna_df=pd.read_csv('../input/kfold-splits-rsna-cq500/RSNA_Validation_f2.csv')

In [ ]:
rsna_df=pd.DataFrame(rsna_df.loc[i][1:] for i in range(0,len(rsna_df)) )

In [ ]:
rsna_df

In [ ]:
rsna_df2=pd.read_csv('../input/cq500-normal-images-and-labels/NormalAbnormal/NormalAbnormal/RSNA/Validation_f2.csv')

In [ ]:
rsna_df2

add normal data to rsna dataframe

In [ ]:
rsna_df2=pd.DataFrame(rsna_df2.loc[i][1:2] for i in range(0,len(rsna_df2)) if rsna_df2.loc[i]['Normal'].all()==True)
rsna_df2['epidural']=0
rsna_df2['intraparenchymal']=0
rsna_df2['intraventricular']=0	
rsna_df2['subarachnoid']=0	
rsna_df2['subdural']=0
rsna_df2['normal']=1

rsna_df2

In [ ]:
rsna_df['normal']=0

frames = [rsna_df, rsna_df2]
rsna_df = pd.concat(frames, ignore_index=True)
rsna_df=shuffle(rsna_df)

In [ ]:
rsna_df

In [ ]:


print(len(rsna_df))

HEIGHT = 256
WIDTH = 256
CHANNELS = 3
VALID_BATCH_SIZE = 64
SHAPE = (HEIGHT, WIDTH, CHANNELS)

class_names = ['epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural','normal']
data_generator_test = TrainDataGenerator(rsna_df,
                                        class_names, 
                                        VALID_BATCH_SIZE, 
                                        SHAPE,
                                        augment = False,
                                        shuffle = False
                                        )


y_true = rsna_df[class_names].values
y_hat = model.predict(data_generator_test, verbose=1)

y_hat = y_hat[0:len(y_true)]
y_pred = np.round(y_hat)

In [ ]:
plt.imshow(data_generator_test[5][0][1])

In [ ]:
(data_generator_test[6][0][0])

results for testing on rsna dataframe

In [ ]:
print_metrics_table(y_true, y_hat, y_pred, class_names)

In [ ]:
print_precision_recall_curves(y_true, y_hat, y_pred, class_names)

In [ ]:
print_auc_curves(y_true, y_hat, y_pred, class_names)

In [ ]:
print_confusion_matrix(y_true, y_hat, y_pred, class_names)

In [ ]:
print_on_vs_all_cmatrix(y_true, y_hat, y_pred, class_names)